# Random Forests: Presidential Contributions

Let's look at a random forests models for the presidential dataset.

This dataset defines all presidential contribution amounts from publicly available information.

The purpose here is to try to classify the candidate to whom the contributor contributes.  

Here are the feature columns we will use:
1. Last Name (converted from Contributor Name)
2. First Name (converted from Contributor Name)
3. State 
4. Latitude (converted from Zipcode)
5. Longitude (converted from zipcode)
6. Employer
7. Occupation

### Notes

This is going to be a very difficult dataset to get high accuracy, because we don't have any features that are highly correlated with the outcome. Part of our analysis is to see which features prove to be the most useful. 

One might suspect that information like State, might be very predictive -- because presumably New Yorkers might contribute to Hillary Clinton and Texans might contribute to Donald Trump. However, it turns out that State is pretty weakly correlated to the outcome.  

One nice thing about random forests is that since we "bag" featues in differnet trees, we can empirically see which variables have hte most predictive power.  This is helpful for analytical reasons.



In [3]:
%matplotlib inline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import isnan, when, count, col, split, trim, countDistinct, abs 
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType

import pyspark.sql.functions

print('Spark UI running on http://YOURIPADDRESS:' + sc.uiWebUrl.split(':')[2])

Spark UI running on http://YOURIPADDRESS:4041


## Step 1: Load the data

In [4]:
dataset = spark.read.csv("/data/presidential_election_contribs/2016/2016-medium-clean.csv", header=True, inferSchema=True)


In [5]:
prediction_column = ['CAND_NM']
numeric_columns = ['LAT', 'LNG']
feature_columns = ['LASTNAME', 'FIRSTNAME', 'CONTBR_ST', 'LAT', 'LNG', 'CONTBR_EMPLOYER', "CONTBR_OCCUPATION"]
categorical_columns = ['LASTNAME', 'FIRSTNAME', 'CONTBR_ST', 'CONTBR_EMPLOYER', "CONTBR_OCCUPATION"]
categorical_index = ['FIRSTNAME_index', 'LASTNAME_index', 'CONTBR_ST_index', 'CONTBR_EMPLOYER_index', 
                     "CONTBR_OCCUPATION_index"]

**=> TODO: Print out a count broken down by candidate? **

In [6]:
# Print out a grouping by candidate
# TODO: Print Breakdown.

**=> Which candidates got the most donations? (in terms of number of donors) **

## Step 2: Build Indexers and feature vector

Let's index all the categorical columns, and build a labeld index.

In [7]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep").fit(dataset) for column in categorical_columns ]
pipeline = Pipeline(stages=indexers)
df_r2 = pipeline.fit(dataset).transform(dataset)

**=> TODO: Build vectors from all the numeric and categorical_index columns **

**HINT: numeric_columns and categorical_index**

In [8]:
assembler2 = VectorAssembler(inputCols=numeric_columns + categorical_index, outputCol="features")
fv2 = assembler2.transform(df_r2.na.drop())


In [9]:

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="CAND_NM", outputCol="indexedLabel").fit(fv2)


# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer2 =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(fv2)



## Step 3: Train Random Forest Model with Pipeline

Train the model here.

**=> TODO: Create pipeline with labelIndexer, featureIndexer2, rf2 **



In [10]:

# Train a RandomForest model.
rf2 = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=20, maxBins=10000)

# Chain indexer and forest in a Pipeline
pipeline2 = Pipeline(stages=[labelIndexer, featureIndexer2, rf2])



## Step 4: Split data into training and test

**=> TODO: build training and test datasets 70%/30% **


In [12]:

# Split the data into training and test sets (30% held out for testing)
(trainingData2, testData2) = fv2.randomSplit([.7,.3]) # do a random split 70%/30%


## Step 5: Train the Model

**=> TODO: Get predictions by transforming testData2 **



In [13]:
# Train model.  This also runs the indexers.


model2 = pipeline2.fit(trainingData2)

# Make predictions.
predictions2 = model2.transform(testData2)  #Transform test data


In [14]:
predictions2.select(feature_columns + ['probability', 'prediction']).show()

+--------------+----------+---------+---------+-----------+--------------------+--------------------+--------------------+----------+
|      LASTNAME| FIRSTNAME|CONTBR_ST|      LAT|        LNG|     CONTBR_EMPLOYER|   CONTBR_OCCUPATION|         probability|prediction|
+--------------+----------+---------+---------+-----------+--------------------+--------------------+--------------------+----------+
|        BABYAK|   MICHAEL|       DC|38.905026| -76.983607|             RETIRED|             RETIRED|[0.09932268783622...|       2.0|
|         HAGUE|  PATRICIA|       CA|37.814613|-121.990552|             RETIRED|             RETIRED|[0.13549143212822...|       2.0|
|        BENACK|  MARY ANN|       WA| 47.63714|-122.321891|           MICROSOFT|    CUSTOMER SERVICE|[0.42783058780240...|       0.0|
|      DELAROSA|     PEDRO|       FL|25.753332| -80.270379|       SELF-EMPLOYED|      MEDICAL DOCTOR|[0.49300604784054...|       0.0|
|         GROTH|     LAURA|       DC|38.914363| -77.079339|   

## Step 6: Evaulate the model

In [15]:

# Select example rows to display.
predictions2.select("prediction", "CAND_NM", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator2 = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator2.evaluate(predictions2)
print("Test Error = %g" % (1.0 - accuracy))

rfModel2 = model2.stages[2]
print(rfModel2)  # summary only

+----------+--------------------+--------------------+
|prediction|             CAND_NM|            features|
+----------+--------------------+--------------------+
|       2.0|    Trump, Donald J.|[38.905026,-76.98...|
|       2.0|    Trump, Donald J.|[37.814613,-121.9...|
|       0.0|Clinton, Hillary ...|[47.63714,-122.32...|
|       0.0|Clinton, Hillary ...|[25.753332,-80.27...|
|       0.0|Clinton, Hillary ...|[38.914363,-77.07...|
+----------+--------------------+--------------------+
only showing top 5 rows

Test Error = 0.483467
RandomForestClassificationModel (uid=RandomForestClassifier_4a21987b15aedb7355e5) with 20 trees


**=> TODO: Think about the test error here?  Does it seem high?  What does that say about our model?**

**=> How do we define model success? **

### Decoding the Label

0. Hillary Clinton
1. Bernie Sanders
2. Donald Trump
3. Ted Cruz

## Step 7: Print out the Confusion Matrix


In [ ]:
predictions2.groupBy('CAND_NM').pivot('prediction', range(0,22)).count().na.fill(0).orderBy('CAND_NM').show()

Use the list above to interpret the label.  

**=>What can you conclude from the confusion matrix?**

Is our model better at predicting candidates with many donations (Clinton, Sanders), or few donations?

What can you say about our model perfromance.

## Step 8: Print the feature importanes

In [ ]:
rfModel2.featureImportances

In [ ]:
print(numeric_columns + categorical_columns_donation) # for reference

**=> TODO Compare the relative weight of the feature importances? **



## Conclusion: Most important Fields

1. Employer
2. Occupation
3. LastName
4. State

Other fields not significant

**=> TODO Compare the relative weight of the feature importances? **

Why do you think that the lat/long and other fields did not contribute?

**=> BONUS: Do a Pearson Correlation Matrix of the variables to the outcome, to see correlation **

